In [85]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [86]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-nanum is already the newest version (20200506-1).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 12 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/usr/share/fonts/truetype/

In [87]:
import pandas as pd
import numpy as np

import os, re

from tqdm import tqdm
from konlpy.utils import pprint
from konlpy.tag import Okt
import torchtext

okt = Okt()

# 경고문구 미표시
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 지정
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')

In [88]:
print(okt)

In [89]:
!pip install konlpy

# 티처빌 데이터 셋


In [90]:
t_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/천재교육 프로젝트/티처빌_2022년.csv')
t_df

,Unnamed: 0,0,1,2,3,4,5
0,0,1학점,"유쾌한 미술, 베스트 15 - 일당백","이번 연수, 유쾌한 미술 제목처럼 유쾌하고 유익한 시간이었습니다",5,인문·교양,2023/10/23
1,1,4학점,만물박사로 거듭나는 아하 그렇구나!,본 수업은 우리 주변 생활과 밀접하게 연관 되어진 수업이었습니다.\n다양한 강사분의...,5,인문·교양,2023/10/23
2,2,4학점,미술로 만나는 아이들의 심리,미술치료에 관한 다양한 사례를 설명들을 수 있었습니다. 특히 초등단계에서 적용할 수...,5,생활지도,2023/10/23
3,3,2학점,SPEAK UP GRAMMAR,학창시절 오랜 기간 동안 영어 공부를 하면서 배웠던 문법이지만 그 당시는 이해하기 ...,5,어학,2023/10/23
4,4,1학점,"미래교육의 첫 걸음, 학교자치","앞으로 교육청이나 교육부의 정해진 교육과정이 아니라 단위학교에서 교사,학생, 학부...",5,생활지도,2023/10/23
...,...,...,...,...,...,...,...
68551,68551,1학점,풀꽃 시인 나태주와 함께하는 시간의 쉼표,나태주 시인의 좋은 시 감상과 함께 풍부한 이해를 할 수 있는 좋은 기회였습니다.\...,5,인문·교양,2022/10/13
68552,68552,1학점,긍정심리 접근을 통해 행복을 키워주는 상담 기법,이 강의를 듣고나니 이제는 학급운영을 더 잘할 수 있다는 자신감이 생겼어요. 많은 ...,5,진로·인성,2022/10/13
68553,68553,1학점,현장 멘토들과 함께 하는 인도주의 실천 가이드,이 강의를 듣고나니 이제는 학급운영을 더 잘할 수 있다는 자신감이 생겼어요. 많은 ...,5,인문·교양,2022/10/13
68554,68554,1학점,현장 멘토들과 함께 하는 인도주의 실천 가이드,이 강의를 듣고나니 이제는 학급운영을 더 잘할 수 있다는 자신감이 생겼어요. 많은 ...,5,인문·교양,2022/10/13


In [91]:
#unamed 인덱스 drop
t_df.drop(['Unnamed: 0'], axis =1, inplace=True)
t_df

,0,1,2,3,4,5
0,1학점,"유쾌한 미술, 베스트 15 - 일당백","이번 연수, 유쾌한 미술 제목처럼 유쾌하고 유익한 시간이었습니다",5,인문·교양,2023/10/23
1,4학점,만물박사로 거듭나는 아하 그렇구나!,본 수업은 우리 주변 생활과 밀접하게 연관 되어진 수업이었습니다.\n다양한 강사분의...,5,인문·교양,2023/10/23
2,4학점,미술로 만나는 아이들의 심리,미술치료에 관한 다양한 사례를 설명들을 수 있었습니다. 특히 초등단계에서 적용할 수...,5,생활지도,2023/10/23
3,2학점,SPEAK UP GRAMMAR,학창시절 오랜 기간 동안 영어 공부를 하면서 배웠던 문법이지만 그 당시는 이해하기 ...,5,어학,2023/10/23
4,1학점,"미래교육의 첫 걸음, 학교자치","앞으로 교육청이나 교육부의 정해진 교육과정이 아니라 단위학교에서 교사,학생, 학부...",5,생활지도,2023/10/23
...,...,...,...,...,...,...
68551,1학점,풀꽃 시인 나태주와 함께하는 시간의 쉼표,나태주 시인의 좋은 시 감상과 함께 풍부한 이해를 할 수 있는 좋은 기회였습니다.\...,5,인문·교양,2022/10/13
68552,1학점,긍정심리 접근을 통해 행복을 키워주는 상담 기법,이 강의를 듣고나니 이제는 학급운영을 더 잘할 수 있다는 자신감이 생겼어요. 많은 ...,5,진로·인성,2022/10/13
68553,1학점,현장 멘토들과 함께 하는 인도주의 실천 가이드,이 강의를 듣고나니 이제는 학급운영을 더 잘할 수 있다는 자신감이 생겼어요. 많은 ...,5,인문·교양,2022/10/13
68554,1학점,현장 멘토들과 함께 하는 인도주의 실천 가이드,이 강의를 듣고나니 이제는 학급운영을 더 잘할 수 있다는 자신감이 생겼어요. 많은 ...,5,인문·교양,2022/10/13


In [92]:
t_df.columns

Index(['0', '1', '2', '3', '4', '5'], dtype='object')

In [93]:
t_df.rename(columns = {'0' : '학점',
                       '1' : 'Title',
                       '2' : 'Contents',
                       '3' : 'Star',
                       '4' : 'Domain',
                       '5' : 'Date'}, inplace=True)
t_df

,학점,Title,Contents,Star,Domain,Date
0,1학점,"유쾌한 미술, 베스트 15 - 일당백","이번 연수, 유쾌한 미술 제목처럼 유쾌하고 유익한 시간이었습니다",5,인문·교양,2023/10/23
1,4학점,만물박사로 거듭나는 아하 그렇구나!,본 수업은 우리 주변 생활과 밀접하게 연관 되어진 수업이었습니다.\n다양한 강사분의...,5,인문·교양,2023/10/23
2,4학점,미술로 만나는 아이들의 심리,미술치료에 관한 다양한 사례를 설명들을 수 있었습니다. 특히 초등단계에서 적용할 수...,5,생활지도,2023/10/23
3,2학점,SPEAK UP GRAMMAR,학창시절 오랜 기간 동안 영어 공부를 하면서 배웠던 문법이지만 그 당시는 이해하기 ...,5,어학,2023/10/23
4,1학점,"미래교육의 첫 걸음, 학교자치","앞으로 교육청이나 교육부의 정해진 교육과정이 아니라 단위학교에서 교사,학생, 학부...",5,생활지도,2023/10/23
...,...,...,...,...,...,...
68551,1학점,풀꽃 시인 나태주와 함께하는 시간의 쉼표,나태주 시인의 좋은 시 감상과 함께 풍부한 이해를 할 수 있는 좋은 기회였습니다.\...,5,인문·교양,2022/10/13
68552,1학점,긍정심리 접근을 통해 행복을 키워주는 상담 기법,이 강의를 듣고나니 이제는 학급운영을 더 잘할 수 있다는 자신감이 생겼어요. 많은 ...,5,진로·인성,2022/10/13
68553,1학점,현장 멘토들과 함께 하는 인도주의 실천 가이드,이 강의를 듣고나니 이제는 학급운영을 더 잘할 수 있다는 자신감이 생겼어요. 많은 ...,5,인문·교양,2022/10/13
68554,1학점,현장 멘토들과 함께 하는 인도주의 실천 가이드,이 강의를 듣고나니 이제는 학급운영을 더 잘할 수 있다는 자신감이 생겼어요. 많은 ...,5,인문·교양,2022/10/13


In [94]:
sample_content = t_df['Contents'].iloc[300]
print(type(sample_content))
print(sample_content)

<class 'str'>
학습부진 학생들을 위하여 교과별 구체적인 학습전략이 자세한 예시 설명과 함께 안내되어

무척 유익한 수업이었습니다.


# -----Token Okt-----

In [95]:
#어절만을 뽑았을 때,
p_token = okt.phrases(sample_content)
print(p_token)

['학습부진', '학습부진 학생들', '교과별', '학습전략', '자세한 예시', '자세한 예시 설명', '안내', '무척', '무척 유익한 수업', '학습', '부진', '학생들', '교과', '전략', '예시', '설명', '수업']


In [96]:
token = okt.pos(sample_content)
pprint(token)

[('학습', 'Noun'),
 ('부진', 'Noun'),
 ('학생', 'Noun'),
 ('들', 'Suffix'),
 ('을', 'Josa'),
 ('위', 'Noun'),
 ('하여', 'Verb'),
 ('교과', 'Noun'),
 ('별', 'Noun'),
 ('구체', 'Noun'),
 ('적', 'Suffix'),
 ('인', 'Josa'),
 ('학습', 'Noun'),
 ('전략', 'Noun'),
 ('이', 'Josa'),
 ('자세한', 'Adjective'),
 ('예시', 'Noun'),
 ('설명', 'Noun'),
 ('과', 'Josa'),
 ('함께', 'Adverb'),
 ('안', 'VerbPrefix'),
 ('내', 'VerbPrefix'),
 ('되어', 'Verb'),
 ('\n\n', 'Foreign'),
 ('무척', 'Noun'),
 ('유익한', 'Adjective'),
 ('수업', 'Noun'),
 ('이었습니다', 'Verb'),
 ('.', 'Punctuation')]


In [97]:
tokens = okt.morphs(sample_content)
pprint(tokens)

['학습',
 '부진',
 '학생',
 '들',
 '을',
 '위',
 '하여',
 '교과',
 '별',
 '구체',
 '적',
 '인',
 '학습',
 '전략',
 '이',
 '자세한',
 '예시',
 '설명',
 '과',
 '함께',
 '안',
 '내',
 '되어',
 '\n\n',
 '무척',
 '유익한',
 '수업',
 '이었습니다',
 '.']


In [98]:
tokens = okt.nouns(sample_content)
tokens = okt.nouns(sample_content)
print(tokens)

['학습', '부진', '학생', '위', '교과', '별', '구체', '학습', '전략', '예시', '설명', '무척', '수업']


In [99]:
review_data = t_df['Contents'].dropna().values
review_data = review_data[:68556]

print(review_data.shape)
print(review_data[0])


(68556,)
이번 연수, 유쾌한 미술 제목처럼 유쾌하고 유익한 시간이었습니다


In [100]:
cleaned_content_data = []

for review in tqdm(review_data):
  n_tokens = okt.nouns(review)
  cleaned_tokens = []
  for word in n_tokens:
    if len(word) > 1:
      cleaned_tokens.append(word)
    else:
      pass
  cleaned_content = " ".join(cleaned_tokens)
  cleaned_content_data.append(cleaned_content)

print(len(cleaned_content_data))
print(cleaned_content_data[0])

100%|██████████| 68556/68556 [10:33<00:00, 108.24it/s]

68556
이번 연수 미술 제목 시간


In [101]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfid = TfidfVectorizer()

review_tfid = tfid.fit_transform(cleaned_content_data)

print(review_tfid.shape)

print(review_tfid[0])

(68556, 13649)
  (0, 6700)	0.3346308258785312
  (0, 10243)	0.6179327640170019
  (0, 4192)	0.515071271850297
  (0, 7852)	0.18643680303909052
  (0, 8976)	0.45400904496853484


In [102]:
vocab = tfid.vocabulary_

print(len(vocab))

print({ k:v for i, (k, v) in enumerate(vocab.items()) if i < 30 })

13649
{'이번': 8976, '연수': 7852, '미술': 4192, '제목': 10243, '시간': 6700, '수업': 6393, '우리': 8337, '주변': 10524, '생활': 5800, '연관': 7806, '강사': 282, '설명': 5973, '생명': 5772, '안전': 7224, '지식': 10772, '화재': 13347, '발생': 4455, '원자력': 8490, '발전소': 4468, '관해': 1043, '활용': 13420, '내용': 1980, '관련': 1009, '유튜브': 8710, '보고': 4812, '조선시대': 10343, '실학자': 6949, '다산': 2279, '정약용': 10161, '선생': 5924}


In [103]:
# 단어들의 사전 인덱스를 이용하여 원래 단어를 검색하는 매핑 딕셔너리
index_to_word = { v:k for k, v in vocab.items()}

# 앞에서 5개의 단어를 출력
print({  k:v for i, (k, v) in enumerate(index_to_word.items()) if i < 5 })

{8976: '이번', 7852: '연수', 4192: '미술', 10243: '제목', 6700: '시간'}


In [104]:
# 사이킷런 패키지 활용
from sklearn.decomposition import LatentDirichletAllocation

# LDA 모델링 객체를 생성 (토픽 갯수를 2로 설정: 긍정/부정)
lda = LatentDirichletAllocation(n_components=2)

# TF-IDF 벡터를 입력하여 모델 학습
lda.fit(review_tfid)

LatentDirichletAllocation(n_components=2)

In [105]:
print(lda.components_.shape)

(2, 13649)


In [106]:
for idx, topic in enumerate(lda.components_):
  print(f'토픽 유형 {idx+1}:', [(index_to_word[i], topic[i].round(3)) for i in topic.argsort()[:-11:-1]])

토픽 유형 1: [('연수', 1851.772), ('강의', 1583.725), ('설명', 1013.684), ('내용', 956.074), ('시간', 931.231), ('생각', 821.894), ('영어', 791.41), ('다시', 662.946), ('선생님', 651.382), ('공부', 646.84)]
토픽 유형 2: [('연수', 3085.758), ('도움', 2662.046), ('학생', 2036.057), ('수업', 1959.771), ('활용', 1488.984), ('내용', 1376.604), ('교육', 1346.702), ('아이', 1194.009), ('이해', 1171.225), ('학교', 1093.184)]


In [24]:
!pip install pyLDAvis

In [107]:
stars = t_df['Star'].iloc[:68556].values
print(stars.shape)
print(stars[:5])


(68556,)
[5 5 5 5 5]


In [108]:
test_review_tfid = tfid.transform([cleaned_content_data[0]])
print(test_review_tfid)

  (0, 10243)	0.6179327640170019
  (0, 8976)	0.45400904496853484
  (0, 7852)	0.18643680303909052
  (0, 6700)	0.3346308258785312
  (0, 4192)	0.515071271850297


In [109]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()

lr.fit(review_tfid, stars)
#1,2,3,4,5

LogisticRegression()

In [110]:
pred = lr.predict(review_tfid[0])
print(pred)

[5]


In [111]:
t_df

,학점,Title,Contents,Star,Domain,Date
0,1학점,"유쾌한 미술, 베스트 15 - 일당백","이번 연수, 유쾌한 미술 제목처럼 유쾌하고 유익한 시간이었습니다",5,인문·교양,2023/10/23
1,4학점,만물박사로 거듭나는 아하 그렇구나!,본 수업은 우리 주변 생활과 밀접하게 연관 되어진 수업이었습니다.\n다양한 강사분의...,5,인문·교양,2023/10/23
2,4학점,미술로 만나는 아이들의 심리,미술치료에 관한 다양한 사례를 설명들을 수 있었습니다. 특히 초등단계에서 적용할 수...,5,생활지도,2023/10/23
3,2학점,SPEAK UP GRAMMAR,학창시절 오랜 기간 동안 영어 공부를 하면서 배웠던 문법이지만 그 당시는 이해하기 ...,5,어학,2023/10/23
4,1학점,"미래교육의 첫 걸음, 학교자치","앞으로 교육청이나 교육부의 정해진 교육과정이 아니라 단위학교에서 교사,학생, 학부...",5,생활지도,2023/10/23
...,...,...,...,...,...,...
68551,1학점,풀꽃 시인 나태주와 함께하는 시간의 쉼표,나태주 시인의 좋은 시 감상과 함께 풍부한 이해를 할 수 있는 좋은 기회였습니다.\...,5,인문·교양,2022/10/13
68552,1학점,긍정심리 접근을 통해 행복을 키워주는 상담 기법,이 강의를 듣고나니 이제는 학급운영을 더 잘할 수 있다는 자신감이 생겼어요. 많은 ...,5,진로·인성,2022/10/13
68553,1학점,현장 멘토들과 함께 하는 인도주의 실천 가이드,이 강의를 듣고나니 이제는 학급운영을 더 잘할 수 있다는 자신감이 생겼어요. 많은 ...,5,인문·교양,2022/10/13
68554,1학점,현장 멘토들과 함께 하는 인도주의 실천 가이드,이 강의를 듣고나니 이제는 학급운영을 더 잘할 수 있다는 자신감이 생겼어요. 많은 ...,5,인문·교양,2022/10/13


In [112]:
t_df['Star'].value_counts()

Star
5    63866
4     3386
3      963
1      195
2      146
Name: count, dtype: int64

In [58]:
for i in range(len(t_df)):
  test_pred = lr.predict(review_tfid)[i]
  if (test_pred == 4 or test_pred == 5):
    t_df['LABEL'] = 0
    #긍정
  else:
    t_df['LABEL'] = 1
    #부정



# test_pred = lr.predict(test_review_tfid)[0]
# print("분석 결과 {}적인 리뷰로 예측됩니다. ".format("긍정" if test_pred > 4 or 5 else "부정"))

In [63]:
t_dff = t_df.groupby(['Domain'])['Star'].value_counts().sort_values(ascending=False)
pd.DataFrame(t_dff)

,,count
Domain,Star,
생활지도,5,20642
인문·교양,5,15231
학습지도,5,8605
디지털활용,5,8225
어학,5,5071
진로·인성,5,3123
자격대비,5,1408
교과지도,5,1308
생활지도,4,1093


In [61]:
t_df

,학점,Title,Contents,Star,Domain,Date,LABEL
0,1학점,"유쾌한 미술, 베스트 15 - 일당백","이번 연수, 유쾌한 미술 제목처럼 유쾌하고 유익한 시간이었습니다",5,인문·교양,2023/10/23,0
1,4학점,만물박사로 거듭나는 아하 그렇구나!,본 수업은 우리 주변 생활과 밀접하게 연관 되어진 수업이었습니다.\n다양한 강사분의...,5,인문·교양,2023/10/23,0
2,4학점,미술로 만나는 아이들의 심리,미술치료에 관한 다양한 사례를 설명들을 수 있었습니다. 특히 초등단계에서 적용할 수...,5,생활지도,2023/10/23,0
3,2학점,SPEAK UP GRAMMAR,학창시절 오랜 기간 동안 영어 공부를 하면서 배웠던 문법이지만 그 당시는 이해하기 ...,5,어학,2023/10/23,0
4,1학점,"미래교육의 첫 걸음, 학교자치","앞으로 교육청이나 교육부의 정해진 교육과정이 아니라 단위학교에서 교사,학생, 학부...",5,생활지도,2023/10/23,0
...,...,...,...,...,...,...,...
68551,1학점,풀꽃 시인 나태주와 함께하는 시간의 쉼표,나태주 시인의 좋은 시 감상과 함께 풍부한 이해를 할 수 있는 좋은 기회였습니다.\...,5,인문·교양,2022/10/13,0
68552,1학점,긍정심리 접근을 통해 행복을 키워주는 상담 기법,이 강의를 듣고나니 이제는 학급운영을 더 잘할 수 있다는 자신감이 생겼어요. 많은 ...,5,진로·인성,2022/10/13,0
68553,1학점,현장 멘토들과 함께 하는 인도주의 실천 가이드,이 강의를 듣고나니 이제는 학급운영을 더 잘할 수 있다는 자신감이 생겼어요. 많은 ...,5,인문·교양,2022/10/13,0
68554,1학점,현장 멘토들과 함께 하는 인도주의 실천 가이드,이 강의를 듣고나니 이제는 학급운영을 더 잘할 수 있다는 자신감이 생겼어요. 많은 ...,5,인문·교양,2022/10/13,0


In [113]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    |   Package bcp47 is already up-to-dat

True

In [114]:
t_df['Contents'].unique()

array(['이번 연수, 유쾌한 미술 제목처럼 유쾌하고 유익한 시간이었습니다',
       "본 수업은 우리 주변 생활과 밀접하게 연관 되어진 수업이었습니다.\n다양한 강사분의 친절한 설명도 좋았습니다.\n저의 수업에서도 '8장 생명을 지키는 안전 지식'에서 화재 발생 시와 원자력 발전소 안전에 관해 활용해 보았습니다.\n위 내용관 관련된 유튜브를 찾아 보고.. 수업에서 활용해 보았습니다.\n그리고 자세히는 몰랐지만 조선시대 실학자인 다산 정약용 선생에 대해서 아주 상세히 잘 알 수 있었습니다.\n특히, 그가 돌아가실때 남긴 회혼시는 나에게 깊은 감명을 주었습니다.\n그리고 오랜 귀향살이 후 여유당을 짓고 돌아 가실 때 까지 여유당의 뜻 '겨울 냇물을 건너듯, 이웃을 경계하듯, 세상을 조심하며 산다'는 글귀는\n나를 더욱 조심히 주변을 살피며 사는 지혜를 가르쳐 주었습니다. \n이 수업을 들으시면 지식과 지혜가 한단계 높아지는 유익한 수업이 될 것입니다.^^",
       '미술치료에 관한 다양한 사례를 설명들을 수 있었습니다. 특히 초등단계에서 적용할 수 있는 수업지도안과 활동지가 첨부되어 있어서 좋았습니다.',
       ...,
       '이 강의를 듣고나니 이제는 학급운영을 더 잘할 수 있다는 자신감이 생겼어요. 많은 생각도 하게 해주고 저에게 도움이 많이 되는 좋은 강의였습니다.',
       '담임 및 생활안전부 업무 처리에 있어 공감 능력의 필요성을 절실히 느끼고 있었던 바, 이번 연수를 통해 배운 내용을 학교 현장에 적용시킴으로써 많은 도움을 받고 있습니다. 또한, 공감 능력은 일상 생활 속에서 원만한 인간 관계 형성에도 도움이 될 수 있다 생각합니다.',
       '나태주 시인의 좋은 시 감상과 함께 풍부한 이해를 할 수 있는 좋은 기회였습니다.\n\n특히 나태주 시인의 통찰력 있는 말씀은 너무 짧아 아쉬운 느낌도 들었습니다.\n\n나태주 시인의 면담과정을 하나의 파일로 구성해주시면 더욱 좋을 것 같습니다.(^^)'],
    

In [115]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-nanum is already the newest version (20200506-1).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 12 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/usr/share/fonts/truetype/

In [116]:
import matplotlib.font_manager as fm
sys_font = fm.findSystemFonts()

[f for f  in sys_font if 'Nanum' in f]

['/usr/share/fonts/truetype/nanum/NanumSquareRoundR.ttf',
 '/usr/share/fonts/truetype/nanum/NanumGothicBold.ttf',
 '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf',
 '/usr/share/fonts/truetype/nanum/NanumMyeongjo.ttf',
 '/usr/share/fonts/truetype/nanum/NanumBarunGothicBold.ttf',
 '/usr/share/fonts/truetype/nanum/NanumSquareR.ttf',
 '/usr/share/fonts/truetype/nanum/NanumSquareB.ttf',
 '/usr/share/fonts/truetype/nanum/NanumSquareRoundB.ttf',
 '/usr/share/fonts/truetype/nanum/NanumGothicCoding.ttf',
 '/usr/share/fonts/truetype/nanum/NanumMyeongjoBold.ttf',
 '/usr/share/fonts/truetype/nanum/NanumGothicCodingBold.ttf',
 '/usr/share/fonts/truetype/nanum/NanumGothic.ttf']

In [117]:
path = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font_name = fm.FontProperties(fname = path, size=10).get_name()
print(font_name)
plt.rc('font', family=font_name)

NanumBarunGothic


In [118]:
import re
content=[]
for i in range(len(t_df["Contents"])):
    content.append()
filter_content = re.split(' ',str(content))
filter_content

from collections import Counter
count_content = Counter(filter_content)
Counter(filter_content).most_common(300)

from wordcloud import WordCloud, STOPWORDS
wordcloud = WordCloud(width = 400, height= 400,
                      background_color='white',
                      stopwords = set(STOPWORDS),
                      max_words=300,
                      max_font_size = 100,
                      font_path = path
                      ).generate_from_frequencies(count_content)
plt.figure(figsize = (20, 10), facecolor = None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout()
plt.show()

AssertionError: ignored